# [Getting started with the Keras Sequential model](https://keras.io/getting-started/sequential-model-guide/)

<h3>The Sequential model is a linear stack of layers.</h3> 

<p>You can create a Sequential model by passing a list of layer instances to the constructor:</p>

In [63]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Merge

In [64]:
model = Sequential([
    Dense(32, input_dim=784),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])

In [65]:
model

In [66]:
## You can also simply add layers via the .add() method.
#------------------------------------
# model = Sequential()
# model.add(Dense(32, input_dim=784))
# model.add(Activation('relu))
#------------------------------------

### Specifying the input shape

<p>The following three snippets are strictly equivalent:</p>

In [67]:
model = Sequential()
model.add(Dense(32, input_shape=(784,)))

In [68]:
model = Sequential()
model.add(Dense(32, batch_input_shape=(None, 784)))
# Note that the batch dimension is None, so the model will be able to process batches of any size.

In [69]:
model = Sequential()
model.add(Dense(32, input_dim=784))

<p>And so are the following three snippets:</p>

In [70]:
model = Sequential()
model.add(LSTM(32, input_shape=(10, 64)))

In [71]:
model = Sequential()
model.add(LSTM(32, batch_input_shape=(None, 10, 64)))

In [72]:
model = Sequential()
model.add(LSTM(32, input_length=10, input_dim=64))

### The Merge Layer

<p>Here is a model with two separate input branches getting merged:</p>

In [73]:
left_branch = Sequential()
left_branch.add(Dense(32, input_dim=784))

right_branch = Sequential()
right_branch.add(Dense(32, input_dim=784))

merged = Merge([left_branch, right_branch], mode='concat')

final_model = Sequential()
final_model.add(merged)
final_model.add(Dense(10, activation='softmax'))

<p>Such a two-branch model can then be trained like so:</p>

In [74]:
# final_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# final_model.fit([input_data_1, input_data_2], targets)  # pass one data array per model input

<p>You can also pass a function as the mode argument, allowing for arbitrary transformations:</p>

In [75]:
# merged = Merge([left_branch, right_branch], mode=lambda x: x[0] - x[1])

<p>Now you know enough to be able to define <em>almost</em> any model with Keras.</p>
<p>For complex models that cannot be expressed via Sequential and Merge, you can use the functional API.</p>

### Compilation

<p>For a multi-class classification problem:</p>

In [76]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

<p>For a binary classification problem:</p>

In [77]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

<p>For a mean squared error regression problem:</p>

In [78]:
model.compile(optimizer='rmsprop',
             loss='mse')

<p>For custom metrics:</p>

In [79]:
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def false_rates(y_true, y_pred):
    false_neg = "some_result"
    false_pos = "another_result"
    return {
        'false_neg': false_neg,
        'false_pos': false_pos,
    }

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy', mean_pred, false_rates])

### Training

<h4>For training a model, you will typically use the <a href="https://keras.io/models/sequential/#useful-attributes-of-model", target="_blank">fit function</a>.</h4>

<p>For a single-input model with 2 classes (binary):</p>

In [80]:
model = Sequential()
model.add(Dense(1, input_dim=784, activation='sigmoid'))
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [81]:
# Generate dummy data:

import numpy as np
data = np.random.random((1000, 784))
labels = np.random.randint(2, size=(1000, 1))

In [82]:
# Train the model, iterating on the data in batches of 32 samples:

model.fit(data, labels, nb_epoch=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 0.7231 - acc: 0.5270     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 0.7148 - acc: 0.5350     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 0.7106 - acc: 0.5270     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 0.6990 - acc: 0.5360     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 0.6898 - acc: 0.5480     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 0.6904 - acc: 0.5620     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 0.6753 - acc: 0.5810     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 0.6687 - acc: 0.5910     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 0.6622 - acc: 0.5960     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 0.6609 - acc: 0.5930     


<p>For a multi-input model with 10 classes:</p>

In [83]:
left_branch = Sequential()
left_branch.add(Dense(32, input_dim=784))

right_branch = Sequential()
right_branch.add(Dense(32, input_dim=784))

merged = Merge([left_branch, right_branch], mode='concat')

model = Sequential()
model.add(merged)
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [84]:
# Generate dummy data:
import numpy as np
from keras.utils.np_utils import to_categorical
data_1 = np.random.random((1000, 784))
data_2 = np.random.random((1000, 784))

# These are integers between 0 and 9
labels = np.random.randint(10, size=(1000, 1))

# Convert the labels to a binary matrix of size(1000, 10) for use with categorical crossentropy
labels = to_categorical(labels, 10)

# Train the model
# We are passing a list of NumPy arrays as training data because the model has 2 inputs
model.fit([data_1, data_2], labels, nb_epoch=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 3.0046 - acc: 0.1010     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 2.6472 - acc: 0.1290     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 2.4689 - acc: 0.1410     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 2.3713 - acc: 0.1790     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 2.2712 - acc: 0.1850     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 2.2102 - acc: 0.2160     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 2.1003 - acc: 0.2850     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 2.0254 - acc: 0.2750     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 1.8692 - acc: 0.3710     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 1.7901 - acc: 0.3650     


<h2>Got all that? Good! Now try some <a href="https://keras.io/getting-started/sequential-model-guide/#examples", target="_blank">Examples</a>.</h2>